In [1]:
import sys

def dijkstra(adj_matrix, start_city, end_city):
    num_cities = len(adj_matrix)
    distances = [sys.maxsize] * num_cities
    visited = [False] * num_cities
    previous = [None] * num_cities

    distances[start_city] = 0

    for _ in range(num_cities):
        min_dist = sys.maxsize
        current_city = None

        for city in range(num_cities):
            if not visited[city] and distances[city] < min_dist:
                min_dist = distances[city]
                current_city = city

        visited[current_city] = True

        if current_city == end_city:
            break

        for neighbor in range(num_cities):
            if (
                not visited[neighbor]
                and adj_matrix[current_city][neighbor] != 0
                and distances[current_city] + adj_matrix[current_city][neighbor] < distances[neighbor]
            ):
                distances[neighbor] = distances[current_city] + adj_matrix[current_city][neighbor]
                previous[neighbor] = current_city

    if distances[end_city] == sys.maxsize:
        return "No path found."

    path = []
    current_city = end_city
    while current_city is not None:
        path.append(current_city)
        current_city = previous[current_city]

    path.reverse()
    return path, distances[end_city]

In [2]:
import random
import matplotlib.pyplot as plt
import numpy as np

num_villes = 100

def generate_weighted_adjacency_matrix(n):
    adjacency_matrix = np.random.rand(n, n)  # Générer une matrice aléatoire entre 0 et 1
    adjacency_matrix *= 100  # Multiplier par une valeur maximale pour représenter les poids des distances
    adjacency_matrix = np.round(adjacency_matrix).astype(int)  # Arrondir les valeurs et les convertir en entiers
    np.fill_diagonal(adjacency_matrix, 0)  # Remplir les diagonales avec des zéros pour représenter les distances de la ville à elle-même
    return adjacency_matrix

matrice = generate_weighted_adjacency_matrix(num_villes)
print(matrice)

route = dijkstra(matrice, 0, 99)
print(route)

[[ 0  8 95 ... 79 72 84]
 [60  0 22 ...  8 55 11]
 [94 94  0 ... 40 72 87]
 ...
 [81 30 97 ...  0 62 73]
 [70 61 42 ... 66  0 68]
 [25 33 20 ... 72 95  0]]


IndexError: list index out of range

In [7]:
import random
import numpy as np
import matplotlib.pyplot as plt

num_villes = 50
NumTruck = 3

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)

validSolutions = []

# initialise list of visited cities
visitedCities = [[] for i in range(NumTruck)]

# Determine initial cities (can't have 2 times the same)
truckJourneys = []

numValidSol = 10

for n in range(numValidSol):
    for i in range(NumTruck):
        initCity = random.randint(1,num_villes)
        while initCity in truckJourneys:
            initCity = random.randint(1,num_villes)
        truckJourneys.append([initCity])
        visitedCities[i].append(initCity)
    print("Initial cities : ", truckJourneys)

    truckJourneysFinished = 0

    while truckJourneysFinished != NumTruck:
        truckJourneysFinished = 0
        for j in range(NumTruck):
            if len(visitedCities[j]) != num_villes:
                curTruck = j
                curCity = truckJourneys[curTruck][-1]
                neighborsCurCity = adj_matrix[curCity]
                closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                CityFound = False
                minValue = 0
                while CityFound == False:
                    if max(neighborsCurCity) == minValue:
                        closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                        if truckJourneys[curTruck][-2] == closerCity:
                            while neighborsCurCity[closerCity] == 0 or closerCity == truckJourneys[curTruck][-2]:
                                closerCity = neighborsCurCity.index(random.choice(neighborsCurCity))
                            truckJourneys[curTruck].append(closerCity)
                            CityFound = True
                        else:
                            closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                            truckJourneys[curTruck].append(closerCity)
                            CityFound = True
                    else:
                        closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > minValue))
                        if closerCity in visitedCities[j]:
                            minValue = neighborsCurCity[closerCity]
                        else:
                            truckJourneys[curTruck].append(closerCity)
                            visitedCities[j].append(closerCity)
                            CityFound = True
        for k in range(NumTruck):
            if len(visitedCities[k]) == num_villes:
                truckJourneysFinished += 1
    
    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path
    
    for j in range(len(visitedCities)):
        print("Truck Journey ", j, " : ", truckJourneys[j])
    
    for j in range(len(visitedCities)):
        print("With Duplicate", len(truckJourneys[j]))

    for j in range(len(visitedCities)):
        print("Without Duplicate", len(set(truckJourneys[j])))
        
    validSolutions.append(truckJourneys)
    visitedCities = [[] for i in range(NumTruck)]
    truckJourneys = []

print(validSolutions)
print(len(validSolutions))

Initial cities :  [[32], [42], [34]]
Truck Journey  0  :  [32, 23, 36, 7, 41, 34, 14, 4, 12, 25, 43, 8, 29, 46, 18, 33, 11, 40, 10, 19, 15, 17, 37, 30, 26, 45, 31, 35, 42, 27, 6, 9, 28, 21, 38, 2, 44, 1, 39, 47, 13, 0, 49, 48, 20, 5, 16, 22, 34, 3, 25, 24, 24, 19, 18, 46, 23, 32]
Truck Journey  1  :  [42, 35, 46, 18, 33, 11, 40, 10, 34, 36, 23, 32, 47, 25, 43, 8, 29, 12, 9, 6, 4, 14, 5, 17, 19, 15, 31, 45, 44, 1, 39, 38, 2, 49, 48, 20, 21, 28, 41, 7, 30, 26, 16, 22, 13, 0, 24, 37, 27, 42, 3, 3, 42]
Truck Journey  2  :  [34, 36, 23, 32, 47, 25, 43, 8, 29, 46, 18, 33, 11, 40, 10, 4, 12, 9, 6, 27, 42, 35, 31, 45, 44, 1, 39, 38, 2, 19, 15, 17, 37, 30, 26, 16, 5, 14, 48, 20, 49, 0, 13, 22, 28, 21, 24, 41, 7, 3, 3, 25, 47, 32, 23, 36, 34]
With Duplicate 58
With Duplicate 53
With Duplicate 57
Without Duplicate 50
Without Duplicate 50
Without Duplicate 50
Initial cities :  [[26], [28], [28]]
Truck Journey  0  :  [26, 30, 17, 19, 15, 46, 18, 33, 11, 40, 10, 34, 36, 23, 32, 47, 25, 43, 8, 29, 35

In [14]:
import random
import numpy as np
import matplotlib.pyplot as plt

num_villes = 50
NumTruck = 3

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)

validSolutions = []
truckJourneys = []
visitedCities = []

initCity = random.randint(1,num_villes)

numValidSol = 10

for n in range(numValidSol):
    for i in range(NumTruck):
        truckJourneys.append([initCity])
    visitedCities.append(initCity)
    print("Initial cities : ", truckJourneys)

    while len(visitedCities) < num_villes:
        for j in range(NumTruck):
            curTruck = j
            curCity = truckJourneys[curTruck][-1]
            neighborsCurCity = adj_matrix[curCity]
            CityFound = False
            non_zero_indices = [i for i, val in enumerate(neighborsCurCity) if val != 0 and i not in visitedCities]
            if all(item in visitedCities for item in non_zero_indices):
                non_zero_count = len([x for x in neighborsCurCity if x != 0])
                probabilities = [0 if x == 0 else 1 / non_zero_count for x in neighborsCurCity]
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
            else:
                non_zero_indices = len([i for i, val in enumerate(neighborsCurCity) if val != 0 and i not in visitedCities])
                probabilities = [1 / non_zero_indices if val != 0 and i not in visitedCities else 0 for i, val in enumerate(neighborsCurCity)]
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
                visitedCities.append(selectedCity)

    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path

    for j in range(len(truckJourneys)):
        print("Truck Journey ", j, " : ", truckJourneys[j])
    
    validSolutions.append(truckJourneys)
    
    truckJourneys = []
    visitedCities = []



Initial cities :  [[8], [8], [8]]
Truck Journey  0  :  [8, 41, 19, 25, 7, 36, 0, 22, 40, 23, 26, 2, 15, 16, 14, 18, 32, 28, 28, 6, 47, 8]
Truck Journey  1  :  [8, 31, 10, 29, 20, 47, 48, 37, 49, 30, 11, 21, 42, 6, 33, 4, 27, 4, 4, 8]
Truck Journey  2  :  [8, 1, 5, 3, 38, 34, 45, 44, 9, 12, 24, 13, 39, 35, 17, 43, 46, 11, 11, 42, 25, 8]
Initial cities :  [[8], [8], [8]]
Truck Journey  0  :  [8, 38, 22, 25, 14, 16, 37, 21, 40, 48, 0, 6, 3, 42, 36, 29, 49, 32, 32, 19, 47, 8]
Truck Journey  1  :  [8, 33, 23, 26, 5, 19, 47, 11, 30, 2, 10, 15, 9, 17, 45, 31, 27, 1, 1, 12, 48, 8]
Truck Journey  2  :  [8, 41, 7, 24, 43, 34, 44, 12, 35, 28, 4, 32, 46, 18, 20, 39, 13, 42, 42, 25, 8]
Initial cities :  [[8], [8], [8]]
Truck Journey  0  :  [8, 0, 10, 3, 32, 39, 17, 24, 40, 7, 48, 26, 41, 19, 13, 38, 29, 9, 9, 45, 26, 25, 8]
Truck Journey  1  :  [8, 15, 21, 34, 23, 14, 16, 2, 30, 44, 5, 22, 1, 25, 11, 28, 3, 33, 33, 27, 6, 47, 8]
Truck Journey  2  :  [8, 4, 45, 49, 46, 37, 36, 43, 27, 20, 6, 35, 18,

In [13]:
a = [0, 0, 1, 2, 3, 4, 5, 0, 5, 6, 10, 2, 5, 8]
b = [2, 3, 4]

non_zero_indices = [i for i, val in enumerate(a) if val != 0 and i not in b]
c = [1 / len(non_zero_indices) if val != 0 and i not in b else 0 for i, val in enumerate(a)]

print(c)


[0, 0, 0, 0, 0, 0.125, 0.125, 0, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]


In [2]:
a = [1, 2, 3, 8, 10, 48, 49]
b = [1, 3, 10, 11]

result = all(item in a for item in b)
print(result)

False
